<a href="https://colab.research.google.com/github/nagusubra/Solar_panel_dust_detection/blob/main/Solar_panel_dust_detection_compression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install libraries and modules

In [1]:
!pip install -q tensorflow-model-optimization
!pip install openpyxl
!pip install xlsxwriter

import xlsxwriter
import openpyxl
from openpyxl import Workbook
from openpyxl.drawing.image import Image

import tempfile
from tensorflow import keras
import tensorflow_model_optimization as tfmot

from tqdm import tqdm_notebook as tqdm
import os
import time
import pickle
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras import datasets, layers, models
import matplotlib.pyplot as plt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 238.9/238.9 KB 16.7 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 KB 12.5 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dense, Flatten, BatchNormalization, Dropout
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import binary_crossentropy
from sklearn.model_selection import cross_val_score
import tensorflow_datasets as tfds

import seaborn as sn
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import datetime

import os
import zipfile

In [3]:
# mounting google drive (if you are using Colab)
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Evaluation function

In [4]:
# Evaluate Model Size
def get_gzipped_model_size(file):
  # Returns size of gzipped model, in bytes.

  _, zipped_file = tempfile.mkstemp('.zip')
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
  return os.path.getsize(zipped_file)


def evaluate_model(model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df


def evaluate_model_without_saving_stats(model, model_path, model_info, val_dataset):

  # Evaluate test accuracy and test loss
  # model = tf.keras.models.load_model(model_path)
  test_loss, test_acc = model.evaluate(val_dataset, verbose=0)

  # Evaluate Model Size
  model_size = get_gzipped_model_size(model_path)

  # Evaluate Inference Time
  startTime = time.time()
  prediction = model.predict(val_dataset)
  executionTime = (time.time() - startTime)/len(val_dataset)

  # Print
  print('\nModel Accuracy:', test_acc*100, '%')
  print("Model Size: %.2f bytes" % (model_size))
  print("Inference Time is: ", executionTime, "s")

  # Build Evalution dataframe
  evulation_dict = {
                      "Evaluation type": "Evualation",
                      "Model Information": model_info,
                      "Accuracy": str(test_acc*100) + " %",
                      "Loss": str(test_loss*100) + " %",
                      "Model Size": str(model_size) + " bytes",
                      "Inference Time": str(executionTime) + " sec"
                    }
  
  evulation_df = pd.DataFrame.from_dict(evulation_dict, orient='index').reset_index()


  return test_acc, model_size, executionTime, evulation_df




# research paper evaluation function
def evaluate_research_paper_with_model_path(model_path):
  # model_path = "/content/models/solnet.hdf5"
  solnet = load_model(model_path, compile=False)
  history = solnet.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

def evaluate_research_paper_with_model(model):
  history = model.history()
  plt.plot(history.history['loss'])
  plt.plot(history.history['acc'])
  plt.title('acc loss vs epoch')
  plt.xlabel('epoch')
  plt.legend(['loss', 'acc'], loc='upper left')
  plt.show()

# Load base model and Evaluate


In [5]:
final_model_path = '/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/solnetOriginal_final_base_model.hdf5'
final_model = tf.keras.models.load_model(final_model_path)

In [6]:
batch_size = 32
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)


Found 1440 files belonging to 2 classes.
Using 1152 files for training.
Found 1440 files belonging to 2 classes.
Using 288 files for validation.


In [7]:
test_acc, model_size, executionTime, evulation_df = evaluate_model_without_saving_stats(final_model, final_model_path, "#0", val_dataset)
print(test_acc)

9/9 [==============================] - 25s 52ms/step

Model Accuracy: 72.56944179534912 %
Model Size: 659509171.00 bytes
Inference Time is:  2.826037565867106 s
0.7256944179534912


In [8]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,72.56944179534912 %
3,Loss,261.79609298706055 %
4,Model Size,659509171 bytes
5,Inference Time,2.826037565867106 sec


# Compression

## One-shot pruning

## Iterative pruning

In [9]:
def iterative_pruning(model, initial_sparsity, final_sparsity, begin_step, end_step, train_images, train_labels, epochs):
  prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude

  # Define model for pruning.
  pruning_params = {
      'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(initial_sparsity=initial_sparsity,
        final_sparsity=final_sparsity, begin_step=begin_step, end_step=end_step, frequency=100)
  }

  pruned_model = prune_low_magnitude(model, **pruning_params)

  # `prune_low_magnitude` requires a recompile.
  optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
  pruned_model.compile(optimizer='adam',
                loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])


  callbacks = [
    tfmot.sparsity.keras.UpdatePruningStep(),
  ]

  pruned_model.fit(train_images, train_labels, epochs=epochs, validation_split=0.1,
                    callbacks=callbacks)

  # Strip pruning wrappers
  stripped_pruned_model = tfmot.sparsity.keras.strip_pruning(pruned_model)

  return pruned_model, stripped_pruned_model

In [10]:
batch_size = 10000
#location = 'dataset/'
location = "/content/drive/MyDrive/Solar_panel_dust_detection/dataset_1"
label_mode = 'binary'
seed = 10 #changed for each fold made manually

class_names = ['clean', 'dirty']
in_size = [227, 227, 3]

tr_dataset_for_loading = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1], 
                                          subset = 'training', batch_size=batch_size, validation_split=.2)

val_dataset_for_loading = image_dataset_from_directory(directory=location, label_mode= label_mode, class_names=class_names,
                                          seed=seed, labels='inferred', image_size=in_size[:-1],
                                          subset = 'validation', batch_size=batch_size, validation_split=.2)


Found 1440 files belonging to 2 classes.
Using 1152 files for training.
Found 1440 files belonging to 2 classes.
Using 288 files for validation.


In [11]:
type(tr_dataset_for_loading)

tensorflow.python.data.ops.batch_op._BatchDataset

In [12]:
tfds_array = tfds.as_numpy(tr_dataset_for_loading)

In [13]:
tfds_array

In [14]:
# x_train_images_list = []
# x_train_labels_list = []

# cnt = 0

# for image, label in tfds.as_numpy(tr_dataset):
#   # print(type(image), type(label), label, len(label))
#   x_train = image
#   x_train_labels = label

#   image_tensor = tf.convert_to_tensor(image)
#   x_train_images_list.append(image_tensor)

#   label_tensor = tf.convert_to_tensor(label)
#   x_train_labels_list.append(label_tensor)

#   # print(type(x_train))
#   # print(len(x_train))
#   # print(x_train.ndim)
#   # print(x_train.shape)
  
#   cnt+=1
#   print(cnt)
#   print('------')

#   break

# x_train_images = tf.convert_to_tensor(x_train_images_list)
# x_train_labels = tf.convert_to_tensor(x_train_labels_list)

In [15]:

x_train_images = None
x_train_labels = None

for image, label in tfds.as_numpy(tr_dataset_for_loading):
  # print(type(image), type(label), label, len(label))
  x_train_images = image
  x_train_labels = label
  
  # print(type(x_train_images))
  # print(len(x_train_images))
  # print(x_train_images.ndim)
  # print(x_train_images.shape)

  
  print('------')

  # break

------


In [16]:
x_test_images = None
x_test_labels = None

for image, label in tfds.as_numpy(val_dataset_for_loading):
  # print(type(image), type(label), label, len(label))
  x_test_images = image
  x_test_labels = label
  print('------')
  # break

------


In [17]:
# frquency = 100
# target sparsity = 50%
# sparisty for each pruning tep = 50%/100 = 0.05%
# Total Number of Training Samples = 1440
# batch size = 32
# epochs = 3

# possible start and end for the iterative pruning schedule
# Number of learning steps = (1440/32)*3 = 135 (15, 120)
# Number of learning steps = (1440/32)*4 = 180 (, )
# Number of learning steps = (1440/32)*5 = 225 (, )
# Number of learning steps = (1440/32)*6 = 270 (, )


# Number of learning steps = (1440/32)*30 = 1350 (135, 1200)

# iterative_pruning(
                  #     model, 
                  #     initial_sparsity = 0, 
                  #     final_sparsity   = 0.9, 
                  #     begin_step       = 0, 600  (~10% of 5625) # pruning early on the pruning schedule to get optimal model size
                  #     end_step         = ?, 4500 (~80% of 5625) # pruning only till 80% so that model can still have enough leanring steps to learn and build networks for the data
                  #     train_images, 
                  #     train_labels, 
                  #     epochs           = 3, no change ? since the Number of Steps per Epoch = (Total Number of Training Samples = ?) / (Batch Size = ?) 
                  # )

pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning(final_model, 0, 0.5, 15, 120, x_train_images, x_train_labels, 3)
# pruned_model_saprsity_50, stripped_pruned_model_saprsity_50 = iterative_pruning(final_model, 0, 0.5, 135, 1200, x_train_images, x_train_labels, 30)


Epoch 1/3


/usr/local/lib/python3.9/dist-packages/keras/backend.py:5612: UserWarning: "`sparse_categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


33/33 [==============================] - 19s 112ms/step - loss: nan - accuracy: 0.5068 - val_loss: nan - val_accuracy: 0.4655
Epoch 2/3
33/33 [==============================] - 2s 60ms/step - loss: nan - accuracy: 0.4981 - val_loss: nan - val_accuracy: 0.4655
Epoch 3/3
33/33 [==============================] - 2s 63ms/step - loss: nan - accuracy: 0.4981 - val_loss: nan - val_accuracy: 0.4655


In [18]:
stripped_pruned_model_saprsity_50.save('stripped_pruned_model_saprsity_50.h5')

In [19]:
def print_model_weights_sparsity(model):

    for layer in model.layers:
        if isinstance(layer, tf.keras.layers.Wrapper):
            weights = layer.trainable_weights
        else:
            weights = layer.weights
        for weight in weights:
            if "kernel" not in weight.name or "centroid" in weight.name:
                continue
            weight_size = weight.numpy().size
            zero_num = np.count_nonzero(weight == 0)
            print(
                f"{weight.name}: {zero_num/weight_size:.2%} sparsity ",
                f"({zero_num}/{weight_size})",
            )


print_model_weights_sparsity(pruned_model_saprsity_50)

conv2d/kernel:0: 0.00% sparsity  (0/34848)
conv2d_1/kernel:0: 0.00% sparsity  (0/614400)
conv2d_2/kernel:0: 0.00% sparsity  (0/884736)
conv2d_3/kernel:0: 0.00% sparsity  (0/1327104)
conv2d_4/kernel:0: 0.00% sparsity  (0/884736)
dense/kernel:0: 0.00% sparsity  (0/37748736)
dense_1/kernel:0: 0.00% sparsity  (0/16777216)
dense_2/kernel:0: 0.00% sparsity  (0/4096)


In [20]:
test_loss_pruned_50, test_acc_pruned_50 = pruned_model_saprsity_50.evaluate(x_test_images,  x_test_labels, verbose=0)

In [31]:
len(x_test_images)

288

In [41]:
type(val_dataset)

tensorflow.python.data.ops.batch_op._BatchDataset

In [43]:
val_dataset_for_loading.cardinality()

<tf.Tensor: shape=(), dtype=int64, numpy=1>

In [21]:
startTime = time.time()
prediction = pruned_model_saprsity_50.predict(x_test_images)
executionTimePruned50 = (time.time() - startTime)/len(x_test_images)

9/9 [==============================] - 0s 21ms/step


In [22]:
pruned_model_size = get_gzipped_model_size('stripped_pruned_model_saprsity_50.h5')

In [23]:
print('\nPruned Model Accuracy:', test_acc_pruned_50*100, '%')
print("Pruned Model Size: %.2f bytes" % (pruned_model_size))
print("Pruned Inference Time is", executionTimePruned50, "s")


Pruned Model Accuracy: 52.77777910232544 %
Pruned Model Size: 244056.00 bytes
Pruned Inference Time is 0.0022500347759988573 s


In [26]:
pruned_test_acc, pruned_model_size, pruned_executionTime, pruned_evulation_df = evaluate_model_without_saving_stats(pruned_model_saprsity_50, 'stripped_pruned_model_saprsity_50.h5', "#1", val_dataset)
print(pruned_test_acc)

9/9 [==============================] - 23s 43ms/step

Model Accuracy: 52.77777910232544 %
Model Size: 244056.00 bytes
Inference Time is:  2.504352781507704 s
0.5277777910232544


In [27]:
pruned_evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#1
2,Accuracy,52.77777910232544 %
3,Loss,nan %
4,Model Size,244056 bytes
5,Inference Time,2.504352781507704 sec


In [25]:
evulation_df

,index,0
0,Evaluation type,Evualation
1,Model Information,#0
2,Accuracy,72.56944179534912 %
3,Loss,261.79609298706055 %
4,Model Size,659509171 bytes
5,Inference Time,2.826037565867106 sec


In [28]:
pruned_model_saprsity_50.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/pruned_model_saprsity_50.h5')
stripped_pruned_model_saprsity_50.save('/content/drive/MyDrive/Solar_panel_dust_detection/final_base_model/stripped_pruned_model_saprsity_50.h5')

## Quantization

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_to_quantize)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.target_spec.supported_types = [tf.float16]
fp16_quantized_tflite_model = converter.convert()

with open('fp16_quantized_tflite_model.tflite', 'wb') as f:
  f.write(fp16_quantized_tflite_model)
interpreter_fp16_quant = tf.lite.Interpreter(model_path=str('fp16_quantized_tflite_model.tflite'))
interpreter_fp16_quant.allocate_tensors()

evaluate_model(interpreter_fp16_quant, 'fp16_quantized_tflite_model.tflite')